# Lesson 3: Reflection and Blogpost Writing

## Setup

In [3]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [4]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [5]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

[autogen.oai.client: 07-12 17:16:26] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [6]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [7]:
print(reply)

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI! Offering top-notch courses created by professionals like Andrew Ng, this platform makes complex concepts simple and accessible to all. From neural networks to deep learning algorithms, you’ll master it all. Whether you're a beginner or an expert, there's something for everyone. Sharpen your skills, land that dream job, or simply satisfy your curiosity. With DeepLearning.AI, the possibilities are endless. Join the AI revolution today!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [8]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

[autogen.oai.client: 07-12 17:16:31] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [9]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI! Offering top-notch courses created by professionals like Andrew Ng, this platform makes complex concepts simple and accessible to all. From neural networks to deep learning algorithms, you’ll master it all. Whether you're a beginner or an expert, there's something for everyone. Sharpen your skills, land that dream job, or simply satisfy your curiosity. With DeepLearning.AI, the possibilities are endless. Join the AI revolution today!

--------------------------------------------------------------------------------
Critic (to Writer):

The blog post "Unveiling the Power of DeepLearning.AI" effectively captures the ess

## Nested chat - there can be several critic agents. each reviewing the content and make updates.

In [10]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


[autogen.oai.client: 07-12 17:37:12] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [11]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

[autogen.oai.client: 07-12 17:37:30] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [12]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

[autogen.oai.client: 07-12 17:37:41] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [13]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

[autogen.oai.client: 07-12 17:37:55] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


## Orchestrate the nested chats to solve the task

In [14]:
def reflection_message(recipient, messages, sender, config):
    """ 
    Generate a message for the reviewer to reflect on the content. 
    """
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [  # 4 chats. 1 for each reviewer and 1 for meta reviewer
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [15]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [16]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI! Offering top-notch courses created by professionals like Andrew Ng, this platform makes complex concepts simple and accessible to all. From neural networks to deep learning algorithms, you’ll master it all. Whether you're a beginner or an expert, there's something for everyone. Sharpen your skills, land that dream job, or simply satisfy your curiosity. With DeepLearning.AI, the possibilities are endless. Join the AI revolution today!

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat...

## Get the summary

In [17]:
print(res.summary)

Title: Master AI with DeepLearning.AI Online Courses by Andrew Ng

Unleash the potential of artificial intelligence through DeepLearning.AI's award-winning online courses. Led by renowned expert Andrew Ng, these AI training programs cover everything from neural networks to cutting-edge algorithms. Whether you're a novice or a pro, these artificial intelligence courses cater to all skill levels. Elevate your expertise, advance your career prospects, or simply satiate your curiosity. Join the AI revolution today with DeepLearning.AI, your gateway to a brighter, AI-fueled future.

Meta Description: Dive into the world of artificial intelligence with DeepLearning.AI. Explore online courses taught by Andrew Ng, covering neural networks and advanced algorithms. Perfect for beginners and experts alike. Start your AI journey now!


In [19]:
# Access and print the cost from the response object
cost = res.cost  # Modify this line based on the actual attribute name
print(f"The cost of the chat session is: {cost}")

The cost of the chat session is: {'usage_including_cached_inference': {'total_cost': 0.001699, 'gpt-3.5-turbo-0125': {'cost': 0.001699, 'prompt_tokens': 1196, 'completion_tokens': 734, 'total_tokens': 1930}}, 'usage_excluding_cached_inference': {'total_cost': 0.0010825000000000001, 'gpt-3.5-turbo-0125': {'cost': 0.0010825000000000001, 'prompt_tokens': 935, 'completion_tokens': 410, 'total_tokens': 1345}}}


In [20]:
# Inspect the structure of the response object
print(dir(res))
print(res)

# Access the cost if it is nested within another attribute
# Example: cost might be within a 'details' attribute or similar
cost = res.details.cost  # Adjust based on the actual structure
print(f"The cost of the chat session is: {cost}")


['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'chat_history', 'chat_id', 'cost', 'human_input', 'summary']
ChatResult(chat_id=None, chat_history=[{'content': '\n        Write a concise but engaging blogpost about\n       DeepLearning.AI. Make sure the blogpost is\n       within 100 words.\n       ', 'role': 'assistant'}, {'content': "Title: Unveiling the Power of DeepLearning.AI\n\nDive into the world of artificial intelligence with DeepLearning.AI! Offering top-notch courses created by professionals like Andrew Ng, this platform makes complex concepts simple and accessible to all. From neural

AttributeError: 'ChatResult' object has no attribute 'details'